# Fake news classification using LSTM

In [3]:
import pandas as pd

In [4]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [5]:
df=pd.read_csv(r'datasets/train.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.shape

(20800, 5)

In [8]:
#Drop nan value
df=df.dropna()

In [9]:
#Get independ dent feature
X=df.drop('label', axis=1)

In [10]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [11]:
X.shape

(18285, 4)

In [12]:
Y=df['label']

In [13]:
Y.shape

(18285,)

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
#vocabulary size
voc_size=5000

# One hot representation

In [16]:
messges=X.copy()

In [17]:
messges.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
#Data preprocessing

from nltk.stem.porter import PorterStemmer

ps=PorterStemmer()
corpus = []

for i in range(0,len(messges)):
    #print(i)
    review = re.sub('^a-zA-Z',' ',messges['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [21]:
corpus

['hous dem aide: didn’t even see comey’ letter jason chaffetz tweet',
 'flynn: hillari clinton, big woman campu - breitbart',
 'truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason: hollywood would love trump bomb north korea lack tran bathroom (exclus video) - breitbart',
 'benoît hamon win french socialist party’ presidenti nomin - new york time',
 'back-channel plan ukrain russia, courtesi trump associ - new york time',
 'obama’ organ action partner soros-link ‘indivisible’ disrupt trump’ agenda',
 'bbc comedi sketch "real housew isis" caus outrag',
 'russian research discov secret nazi militari base ‘treasur hunter’ arctic [photos]',
 'us offici see link trump russia',
 're: yes, paid govern troll social media, blogs, forum websit',
 'major leagu soccer, argentin find home success - new york time',
 'well fargo chief abruptli step - new york time',
 'anonym donor pay $2.5 milli

In [22]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2466, 4460, 2001, 4759, 4885, 4664, 4846, 109, 1418, 15, 2702],
 [3522, 3016, 4855, 1317, 3395, 230, 1245],
 [3684, 2885, 2730, 132],
 [3942, 438, 2317, 3505, 3409, 3919, 1760],
 [4599, 3395, 3330, 3832, 4966, 4883, 3395, 1314, 4528, 4441],
 [785,
  3729,
  4167,
  1105,
  1919,
  4616,
  390,
  4914,
  743,
  4679,
  3979,
  2847,
  4044,
  945,
  1245],
 [3683, 925, 610, 3844, 448, 4419, 3218, 2252, 2436, 2470, 3819],
 [1555, 710, 524, 1063, 3198, 1737, 4616, 2117, 2436, 2470, 3819],
 [514, 4994, 4232, 1579, 3458, 4766, 3783, 3546, 2940, 2798],
 [2452, 4688, 1197, 242, 1540, 1038, 1854, 3848],
 [3471, 4618, 2430, 406, 3444, 203, 989, 3300, 574, 4689, 4167],
 [3409, 1865, 4664, 4766, 4616, 3198],
 [2436, 2759, 4170, 1246, 4785, 1759, 3349, 4253, 2270, 2240],
 [4037, 3572, 152, 434, 4453, 2335, 877, 2436, 2470, 3819],
 [1184, 964, 2492, 2936, 3356, 2436, 2470, 3819],
 [3893, 1355, 285, 802, 3916, 368, 3200, 447, 616, 2280, 3473, 2165],
 [1081, 3503, 369],
 [644, 1679, 1261, 1674, 461

In [23]:
import numpy as np

In [24]:
#befor embeding set fix length 
sent_len = 20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_len)
print(embedded_docs)

[[   0    0    0 ... 1418   15 2702]
 [   0    0    0 ... 3395  230 1245]
 [   0    0    0 ... 2885 2730  132]
 ...
 [   0    0    0 ... 2436 2470 3819]
 [   0    0    0 ... 4235  438 2074]
 [   0    0    0 ... 3720  478 4580]]


In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0, 2466, 4460,
       2001, 4759, 4885, 4664, 4846,  109, 1418,   15, 2702])

In [34]:
#Creating model
from tensorflow.keras.layers import Dropout

embedding_vector_features=40
model= Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
len(embedded_docs),Y.shape

(18285, (18285,))

In [36]:
import numpy as np
X_final=np.array(embedded_docs)
Y_final=np.array(Y)

In [37]:
X_final.shape,Y_final.shape

((18285, 20), (18285,))

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_final,Y_final, test_size=0.33, random_state=42)


# Model training

In [39]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 17ms/step - loss: 0.3361 - accuracy: 0.8478 - val_loss: 0.1936 - val_accuracy: 0.9171
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1411 - accuracy: 0.9459 - val_loss: 0.1884 - val_accuracy: 0.9234
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1012 - accuracy: 0.9613 - val_loss: 0.3010 - val_accuracy: 0.9140
Epoch 4/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0742 - accuracy: 0.9754 - val_loss: 0.2190 - val_accuracy: 0.9198
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0561 - accuracy: 0.9816 - val_loss: 0.2786 - val_accuracy: 0.9181
Epoch 6/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0393 - accuracy: 0.9878 - val_loss: 0.3234 - val_accuracy: 0.9180
Epoch 7/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0300 - accuracy: 0.9902 - val_loss: 0.3421 - val_accuracy: 0.9170

 model preformence accuracy metrics

In [42]:
Y_pred=model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [56]:
threshold = 0.5  # Adjust as needed
Y_pred_binary = (Y_pred > threshold).astype(int)

In [57]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_pred_binary)

array([[3071,  348],
       [ 166, 2450]], dtype=int64)

In [53]:
#confusion_matrix(Y_test,Y_pred)

1.24.3


In [58]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred_binary)

0.9148301574150787